In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../20230507_Messages.csv")
df.head()

,Datetime,Message
0,2023-03-01 14:28:12,Please leave a one line intro when you join an...
1,2023-03-01 14:29:23,I'm Pranjal Mehta. Cofounded ePlane.ai (electr...
2,2023-03-01 14:29:25,"Hey folks,"
3,2023-03-01 14:29:49,I'm Nirant K. ML/LLM Consultant out of BLR. Mo...
4,2023-03-01 14:30:26,Hey


In [3]:
df["Message"][21]

'Looks like [PHONE REMOVED] got a DM from our benevolent dictator 😂, welcome to the group'

In [4]:
# Group by week
df["Datetime"] = pd.to_datetime(df["Datetime"])
df['Week'] = df['Datetime'].dt.isocalendar().week
df['Date'] = df['Datetime'].dt.date

# Group by Date
daily_df = df.groupby('Date').agg({'Message': ' \n '.join}).reset_index()
daily_df = pd.DataFrame(daily_df)
len(daily_df)

# # Group by Week
# weekly_df = df.groupby('Week').agg({'Message': ' \n '.join}).reset_index()
# weekly_df = pd.DataFrame(messages_df)
# print(weekly)
# print(weekly_df["Message"][9])

66

In [5]:
daily_df["wc"] = daily_df["Message"].apply(lambda x: len(x.split()))

In [6]:
daily_df["wc"].describe()

count      66.000000
mean      905.121212
std       725.419359
min         9.000000
25%       283.000000
50%       699.000000
75%      1471.000000
max      2551.000000
Name: wc, dtype: float64

In [79]:
from datetime import date

d = daily_df["Date"][42]

def human_date(d):
    def ordinal(n):
        return "%d%s" % (n, "tsnrhtdd"[((n//10%10!=1)*(n%10<4)*n%10)::4])

    formatted_date = d.strftime(f"{ordinal(d.day)} %B %Y")
    return formatted_date
print(human_date(d))

14th April 2023


## Summarization
Built with Langchain

In [9]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

from functools import lru_cache
# llm = ChatOpenAI(temperature=0)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()

In [10]:
%%time
class SummarizeDay:
    def __init__(self, plain_text):
        self.prompt_template = """This is a chaotic Generative AI Group Chat transcript. Write detailed, exhaustive bullet point recap of topics discussed. Extract COMPLETE URL of web and social links with context. Please organise it into sections, only when needed:

{text}


Use Markdown. Add ## for section titles. TOPICS RECAP:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output
    
    
def summarize(message: str)->str:
    sd = SummarizeDay(message)
    chain_output = sd.summarize_docs()
    summary_text = chain_output["output_text"]
    print(summary_text)
    return summary_text

CPU times: user 57 µs, sys: 1 µs, total: 58 µs
Wall time: 62.9 µs


In [11]:
%%time
daily_df["Summary"] = daily_df["Message"].apply(summarize)

## Introductions
- Members introduce themselves and their backgrounds in AI, ML, and related fields.

## Projects and Startups
- Pranjal Mehta talks about his electric plane startup, ePlane.ai, and his interest in building in GenAI.
- Nirant K works with startups building in ML.
- Kush is running Corp Dev for Glance & InMobi and is interested in seeing Generative AI innovations in the market.
- Pranav leads Peppertype.ai and has been closely working with Gen AI and its applications in content and marketing for the last 2 years.
- Sachin is the founder of IntelLawyer.com, a search engine of case laws, and has previously worked as an RL engineer.
- Amogh is tinkering on a project in generative AI called Youstick.fun, which lets users create fun stickers using their photos.
- Rahul Sundar is a PhD student interested in teaching the science of any underlying problem to machines.
- Parth is a VC at Venture Highway and looks closely at the Applied AI space.
- Anagh from Accel has been workin

## Hackathon Registration
- Participants only need to fill the Google form for the hackathon
- This chat is for everyone interested in seeing what was built during the hackathon
- Register for a hackathon invite to the BLR venue: https://forms.gle/UizUwyKi4bajt6WB7

## GPT Hype
- Discussion about the hype surrounding GPT
- Question about whether it is different from the Google form for the hackathon

## Company Participation
- Discussion about the lack of company participation in the hackathon
- Someone from the company is encouraged to sign up to participate

## Runwayml
- Question about whether anyone has access to Runwayml
- No further discussion or context provided
## OpenAI LLMs and Foundry

- Discussion about whether there are any LLMs by OpenAI that can be deployed on-premises today
- Mention of OpenAI Foundry, which costs $1 million per year and will have on-premises capabilities
- No specific LLMs mentioned that can be deployed on-premises currently

## FLan-T5 by Google

- Me

AI Models and Tools:
- Hugging Face's text-to-video synthesis model: https://huggingface.co/spaces/damo-vilab/modelscope-text-to-video-synthesis
- Reid Hoffmann's essay on GPT4 and AI summary request
- Midjourney for generic imagery and potential API release
- Stable Diffusion vs. Photoshop and Canva vs. Midjourney comparison
- InvokeAI's open source UI for Stable Diffusion and potential API release
- Automatic1111 as a potential future option for CreativeAI
- RunwayML Gen1 and its impressive features
- Comfy UI, a node-based/DAG interface for image generation
- OpenAI's paper on the potential impact of Large Language Models on the Labor Market: https://twitter.com/rubinovitz/status/1637651591191842816?t=g4U4qTnkF-R02mImutwo8w&s=19

Industry and Applications:
- HBR essay on how generative AI will disrupt video platforms: https://hbr.org/2023/03/how-will-generative-ai-disrupt-video-platforms
- Kishore [PHONE REMOVED]'s work in the generative AI space to generate statics for advertisers


## Video on OpenAI's Development Trajectory
- Link: https://www.youtube.com/watch?v=L_Guz73e6fw
- Discussion about OpenAI's development trajectory
- No technical details, but philosophical nuggets shared by Sam Altman
- Talks about the potential of AI and its impact on society
- Mentions the importance of safety and ethical considerations in AI development

## The AI Dilemma Video
- Link: https://www.youtube.com/@aajtakai/videos
- Discussion about the recent blitzscaling of OpenAI and Microsoft in AI advancements
- Video titled "The AI Dilemma" talks about the safety concerns and risks associated with exponential growth of LLMs (large language models)
- Emphasis on the need for ethical considerations and safety measures in AI development
## Deep Hack Demo Day Invite
- Invitation to a demo day
- Sales pitch to share with friends in Bengaluru
- One member wishes they could attend but will be flying out that day

## Jobot, a ChatGPT-powered bot is live at Jovian
- Announcement of a new Ch

## Generative AI Tools and Platforms
- OpenPlayground and Playground.ai are discussed as generative AI experimentation platforms
- OpenPlayground is noted for its ability to experiment with different diffusion models, parameters, inpainting, and base image generation for LLMs
- Playground.ai is noted for its popular prompts and upvoting system
- API access for Playground.ai is expected to be released soon
- The group discusses the strategy of having a Discord bot but not an API, with some suggesting that it may be due to the cost and difficulty of setting up an inference infrastructure
- Vast.ai and Hetzner Cloud are mentioned as cloud computing services for renting GPUs
- Runpod is mentioned as a service for renting out GPUs
- ImagineAPI.dev is noted as an unofficial MJ API

## AI Research and Development
- The AI Index Report by Stanford University is mentioned
- Langchain, an AI-powered language learning platform, is praised
- The group discusses the potential for Indians in India t

...

LangChain and Tenacity API
- LangChain uses Tenacity API
- +1 on using Tenacity API
- LangChain has an API for the idea of kids as superheroes

Andrej Karpathy's Neural Nets Video
- Finished watching Andrej Karpathy's video on Neural Nets
- Highly recommended for anyone starting or in the field
- Link to the video playlist: https://youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ

Image Inpainting and Diffusers
- Tried Img2img models for inpainting on an online image
- Asked for suggestions on using diffusers for inpainting
- Link to Stable Diffusion XL beta: https://stability.ai/blog/stable-diffusion-xl-beta-available-for-api-customers-and-dreamstudio-users

ChatGPT for GitHub Repository
- Saw a Twitter post about ChatGPT for a GitHub repository
- Link to the demo video of Chat with GitHub: https://js.langchain.com/docs/modules/indexes/document_loaders/examples/web_loaders/github
- Link to LangChain API for asking questions on a GitHub repository: https://js.langchain.com/

OpenAI Plugin Store:
- Adept.ai got copied as an in the OpenAI Plugin Store, called Multi-on 

Experience needed:
- People with experience in CLIP, BLIP-2, and VQA are needed for a project 
- Someone with solid experience in multimodal vector similarity search is also needed 

Meeting invite:
- A meeting invite was shared in the WhatsApp group for the project 

Projects:
- Multimodal vector similarity search 
- Personal search 
- Generative AI for solving problems and building multi-modal systems 
- Domain-specific feedback loop and fine-tuning 
- Context-based search with cohere embeddings and OpenAI GPT3.5 for non-English languages 
- Text to video 
- Applications of generative AI in finance 

Resources:
- GitHub repository for the project: https://github.com/marqo-ai/marqo 
- Deepset's Haystack and Milvus for modern DL infra 
- Character.ai for personal search 
- OpenAI's Point-e for converting image/text to 3D model 
- Segment Anything Model for creating an object mask from an imag

OpenAI and Dense Embeddings:
- Currently using OpenAI and dense embeddings
- Planning to move to hybrid soon
- Using similarity search on a flat embedding space to retrieve context

GitHub and LLM:
- Came across a GitHub where someone used PEFT to fine-tune an LLM based on their iMessage chats to impersonate and create a bot that talks like them

Supabase:
- Discussion about Supabase and Postgres performance queries
- Suggestions to check RLS policies and use explain.dalibo.com to visualize queries
- Offer to help with specific links on Github or Discord forum
- Mention of a new branch for Mac release of Automatic1111

ChatGPT:
- Discussion about school kids in India using ChatGPT
- Mention of e2eml.school/transformers.html
- Anecdotal evidence of kids using ChatGPT for school essays

Generative AI Speakers:
- Discussion about having more Amod-esque speakers
- Suggestions for speakers from AI21 Labs, Cohere, and Anthorpic
- Plans to set up an event/session link to register for upcoming

## VectorDB and LlamaIndex
- Discussion on whether LlamaIndex or Langchain support VectorDB with Sources with GPT4 or GPT3.5-Turbo
- Sharing of GitHub link for LlamaIndex demo
- Suggestion to use response.source_nodes to get sources
- Discussion on using evaluation module or regex to get desired sources
- Mention of OpenAI models not citing references
- Sharing of Langchain link for QA with sources example
- Discussion on limitations of Langchain and need to extend chain functions
- Mention of LlamaIndex not having such limitations

## AI Models and Applications
- Discussion on models for aesthetic score of images
- Mention of ResNet and classification model for user-scored images
- Discussion on generating professional-level DSLR photos using AI
- Mention of LAMINI AI library for fine-tuning LLMs to custom domains
- Announcement of upcoming "Learning Transformers/NLP/ML" discussion
- Discussion on D-ID and SadTalker for generating content
- Mention of Whisper models for Hindi transcri

Introduction and greetings
- Members introduce themselves and exchange pleasantries

Generative AI for digital painting
- Discussion on the potential for creating artist-focused digital painting tools using Generative AI
- Current methods involve hacking with automatic tools and using Photoshop to achieve desired results

Large scale dataset creation for LLM model training
- Member asks if anyone has experience creating large scale datasets
OpenAI and Google models:
- Google models have no moat
- OpenAI may have a moat due to early mover advantage and alignment with Microsoft/Azure
- OpenAI's moat may come from government and finance domains
- OpenAI may be able to maintain its lead for 2 to 5 years or more
- Open source models are far behind OpenAI currently, but may catch up eventually
- Network effects may not be a moat in this case
- Microsoft+OpenAI can make inference costs low and onboard other businesses on plugins

Hugging Face:
- StarCoder LLM launched
- Good intro + inference

In [16]:
import re
WINDOW = 1
def extract_urls_with_context(text):
    lines = text.split('\n')
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    urls_with_context = []

    for idx, line in enumerate(lines):
        for match in url_pattern.finditer(line):
            start, end = match.span()
            prev_line = lines[idx - WINDOW] if idx > 0 else ""
            next_line = lines[idx + WINDOW] if idx < len(lines) - 1 else ""
            context = f"{prev_line}\n{line}\n{next_line}".strip()
            urls_with_context.append((match.group(), context))

    return urls_with_context        


class LinksContext:
    def __init__(self, plain_text):
        self.prompt_template = """For the given URL, there is some context. Newlines may or may not be related to the link, but the message in the same link as link is related to the link.
        
{text}
        
Mention URL with context. Single bullet point:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name="gpt-4"), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output

    
def end_note_with_links(message: str)-> str:
    url_groups = extract_urls_with_context(message)
    link_with_description = []
    for ug in url_groups:
        lc = LinksContext(ug[1])
        chain_output = lc.summarize_docs()
        output_text = chain_output["output_text"]
        link_with_description.append(output_text)
    return "\n".join(link_with_description)
        
# message = daily_df["Message"][33]
# print(end_note_with_links(message))
daily_df["EndNote"] = daily_df["Message"].apply(end_note_with_links)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 925fb04d19c23f158708c8884c20bebf in your message.).


In [74]:
class TitleDescription:
    def __init__(self, plain_text):
        self.prompt_template = """For the given discussion, write a short title and description, separate both by \n\n

{text}
        
Return a single valid JSON with 
```
"title":
"description:":
```:"""
# Research with weblinks where relevant EXACTLY ONCE:
        self.PROMPT = PromptTemplate(template=self.prompt_template, input_variables=["text"])
        self.chain = load_summarize_chain(ChatOpenAI(temperature=0, model_name="gpt-4"), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=self.PROMPT, combine_prompt=self.PROMPT)
#         self.chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=self.PROMPT)
        self.docs = self.make_docs(plain_text)
        
    @lru_cache
    def make_docs(self, plain_text: str):
        texts = text_splitter.split_text(plain_text)
        docs = [Document(page_content=t) for t in texts]
        return docs

    def summarize_docs(self):   
        chain_output = self.chain({"input_documents": self.docs}, return_only_outputs=True)
        return chain_output


    
import json    

def get_title_description(summary_text: str)-> dict[str, str]:
    td = TitleDescription(summary_text)
    title_description = td.summarize_docs()
    try:
        fields = json.loads(title_description["output_text"])
    except Exception as e:
        print(title_description["output_text"])
        title_description = td.summarize_docs()
        fields = json.loads(title_description["output_text"])
    return fields

In [57]:
daily_df.iloc[0]

Date                                              2023-03-01
Message    Please leave a one line intro when you join an...
wc                                                       666
Summary    ## Introductions\n- Members introduce themselv...
EndNote    - https://www.youstick.fun/ - a website that a...
Name: 0, dtype: object

In [55]:
# !pip install pytz

In [76]:
%%time
import pytz
from datetime import datetime, time
def get_page_header_date(date_object):
    # Combine the date object with a time object and set the desired timezone
    dt = datetime.combine(date_object, time())
    desired_timezone = pytz.timezone("Asia/Kolkata")
    localized_dt = desired_timezone.localize(dt)

    # Format the datetime object using strftime
    formatted_datetime = localized_dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    formatted_datetime = formatted_datetime[:-2] + ':' + formatted_datetime[-2:]

    return formatted_datetime
    
def make_page_header(row):
    date, summary_text = row["Date"], row["Summary"]
    dt = get_page_header_date(date)
    fields = get_title_description(summary_text)
    summary_title, summary_description = fields["title"], fields["description"]
    
    page_header = f"""+++
title =  "{summary_title}"
date = {dt}
tags = ["daily_summary"]
featured_image = ""
description = "{summary_description}"
toc = true
+++
"""
    return page_header

page_headers = []
for idx in range(len(daily_df)):
    page_headers.append(make_page_header(daily_df.iloc[idx]))
    print(page_headers[-1])
# daily_df["page_header"] = [make_page_header(row=df_row) for df_row in daily_df.iloc[]]

+++
title =  "Generative AI: Applications, Projects, and Investments"
date = 2023-03-01T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "In this discussion, participants introduce themselves and their backgrounds in AI, ML, and related fields. They share their projects and startups, which range from electric planes to content generation and AI investments. Topics discussed include creating anime with StyleGAN, building agents for gaming and stock trading, generative AI in content and marketing, and recommendation systems for generative media consumption."
toc = true
+++

+++
title =  "Exploring ChatGPT API, Whisper Updates, and Anime Production"
date = 2023-03-02T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "A conversation about the ChatGPT API launch, improvements in Whisper API, controlnet integration, gpt-3.5-turbo's performance in applications, and Corridor Digital's innovative anime creation process using SD VFX and Blender."


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


+++
title =  "AI Bot Code, Art, Purchase Parity, and Movie Ideas"
date = 2023-03-04T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "The group chat discusses the open-sourced Hasura AI bot code, the impact of Offset Noise on AI art, the concept of purchase parity, and a potential AI-themed movie idea."
toc = true
+++

+++
title =  "Exploring the World of Generative AI"
date = 2023-03-05T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "A discussion about the potential, applications, and ethical implications of generative AI, along with sharing personal projects, tools, resources, events, and communities related to the field."
toc = true
+++

+++
title =  "Exploring 3D Art and AI: RealFusion, NeRF, and Embedding Techniques"
date = 2023-03-06T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "Discuss the advancements in 3D art and AI, including Oxford University's RealFusion diffusion model for 360-degree object r

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


+++
title =  "Chaotic Generative AI Group Chat: Art Generation Workflow, AI Tools, and Learning"
date = 2023-03-25T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "The discussion covers using Stable Diffusion for AI art generation, the workflow involving text2img, controlnet depth maps, and Automatic1111. Participants also discuss AI tools like Langchain agents and ChatGPT plugins, few-shot learning, and the concept of seeds in random number generation. Humor and social links are also shared."
toc = true
+++

+++
title =  "OpenAI's Development Trajectory and the AI Dilemma"
date = 2023-03-26T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "A discussion on OpenAI's development trajectory, the potential of AI and its impact on society, and the importance of safety and ethical considerations in AI development. The video also covers the recent blitzscaling of OpenAI and Microsoft in AI advancements, safety concerns and risks associated wi

+++
title =  "Comparing AI Embedding Models and Exploring Applications"
date = 2023-04-12T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "An in-depth comparison of OpenAI's 'text-embedding-ada-002' with Huggingface models and Sentence Transformers, discussing their use in news media, chatroom plugins, and conversational memory. The conversation also covers AI regulation, applications in agriculture and finance, and dataset recommendations, with a focus on Langchain and OpenAI resources."
toc = true
+++

+++
title =  "Dolly 2.0, Consistency Models, Vector Databases, and Cloud Providers"
date = 2023-04-13T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "In this discussion, participants explore the release of Dolly 2.0, a commercially viable LLM by Databricks, OpenAI's consistency models, and various vector databases. They also delve into the APIs of cloud providers like Azure, AWS, and GCP, as well as the future of LLMs, chip supply, c

+++
title =  "AI in Music, Tools, Projects, and Discussions"
date = 2023-04-30T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "Explore the collaboration between GPT and Bark for music creation, AI tools and libraries like pandas-ai, issues with ChatOpenAI, and code snippets for language chain helpers and embeddings utils. Discuss AI models and projects like Cohere's multilingual model, automating tasks, creating charts, and implementing file search. Engage in AI discussions on limited RAM, measuring recall, tuning algorithms, and the impact of generative AI on graphic design and marketing. Discover AI resources like LLM.report, ann-benchmarks, and stable diffusion in graphic design."
toc = true
+++

+++
title =  "AI Integration, Web Development, and Group Management"
date = 2023-05-01T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "A comprehensive discussion on GPT plugins, web development, AutoGPT, managing a GenAI group, AI philos

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


+++
title =  "Exploring ChatGPT, APIs, and Language Models"
date = 2023-05-02T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "An in-depth conversation covering a range of topics such as biased articles on Twitter, learning with ChatGPT, API level caps, paywalls, conversational memory, Langchain, rolling windows, summarization techniques, and additional resources."
toc = true
+++

+++
title =  "Generative AI Group Chat Highlights"
date = 2023-05-03T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "An overview of key topics covered in a group chat centered on generative AI, featuring discussions on AI models, tools, applications, industry insights, networking opportunities, and miscellaneous subjects such as AI in art and startup ecosystems."
toc = true
+++

+++
title =  "Generative AI in Digital Painting and Large Scale Dataset Creation"
date = 2023-05-04T00:00:00+05:30
tags = ["daily_summary"]
featured_image = ""
description = "Member

In [78]:
daily_df["page_headers"] = page_headers
daily_df.to_json("daily_backup.json")
daily_df.head()

,Date,Message,wc,Summary,EndNote,page_headers
0,2023-03-01,Please leave a one line intro when you join an...,666,## Introductions\n- Members introduce themselv...,- https://www.youstick.fun/ - a website that a...,"+++\ntitle = ""Generative AI: Applications, Pr..."
1,2023-03-02,https://twitter.com/OpenAI/status/163099240654...,84,## ChatGPT API Launch and Updates\n- OpenAI pr...,- OpenAI's Twitter URL (https://twitter.com/Op...,"+++\ntitle = ""Exploring ChatGPT API, Whisper ..."
2,2023-03-03,The new Flan-UL2 20B from GoogleAI is out! \n...,721,Generative AI:\n\n- GoogleAI's new Flan-UL2 20...,- Two interesting a16z articles: \n https://a...,"+++\ntitle = ""Exploring Generative AI and Bra..."
3,2023-03-04,https://theresanaiforthat.com/ \n We open-sour...,67,## AI Bot Code\n\n- The group chat mentions th...,- The website https://theresanaiforthat.com/ h...,"+++\ntitle = ""AI Bot Code, Art, Purchase Pari..."
4,2023-03-05,"Hi everyone, I'm Anirudth, currently an Applie...",28,## Introduction\n- Anirudth introduced himself...,,"+++\ntitle = ""Exploring the World of Generati..."


In [86]:
def make_page(row):
    page = row["page_headers"] + "\n" + row["Summary"] + "\n" + "\n## Links\nThe description and link can be mismatched because of extraction errors.\n\n" + row["EndNote"]
    file_name = f"{human_date(row['Date'])}.md"
    return page, file_name

from pathlib import Path
write_dir = Path("../../content/ai/").resolve()

for idx in range(len(daily_df)):
    page, file_name = make_page(daily_df.iloc[idx])
    file_path = write_dir / file_name
    with file_path.open("w") as f:
        f.write(page)
#     break